# Assignment 8: BHC Analysis using Python
#### Group: Mitchell Kramer, Nicholas Meyer,  Nikita Tatarinov, Glenn Matlin, Michael Galarnyk

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
from tqdm import tqdm

# Feel free to add or remove libraries as you see fit
from scipy.stats import skew, kurtosis
from scipy.stats import ttest_1samp
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import acovf
from statsmodels.stats.sandwich_covariance import cov_hac

## Assignment Description

The goal of the assignment is to understand banks financial statements and the risks they are
exposed to. There are a number of issues (listed below) that I want the students to address
in the assignment report. Each group is expected to pick a bank or bank holding company.

1. Draw a random number generated using one of the team member gtid as a seed over theinterval (1, 25). List the seed and your random number at the top of the report.
2. You can select the random number using Python
3. Select the corresponding bank (rank based on size that is the same as the random number) at https://www.ffiec.gov/npw/Institution/TopHoldings
4. If the BHC is not publicly listed in the U.S., redraw the random number
5. I will cold call students in class to describe their findings about their assigned bank in the class

In [2]:
# Dictionary provided by the user
group_id = {
    "Glenn Matlin": 903738044,
    "Nikita Tatarinov": 903914813,
    "Michael Galarnyk": 903918895,
    "Mitchell Kramer": 903176941,
    "Nicholas Meyer": 903444783
}

# Initialize a list to hold the random numbers
random_numbers = []

# Iterate over the GTIDs in the group_id dictionary
# For each GTID, we set it as the seed and generate a random number
for name, gtid in group_id.items():
    np.random.seed(gtid)  # Set the seed
    random_num = np.random.randint(1, 26)  # Generate the random number
    random_numbers.append(random_num)  # Append to the list

# Create a DataFrame from the group_id dictionary and the list of random numbers
df = pd.DataFrame(list(group_id.items()), columns=['Student', 'GTID'])
df['Random_Number'] = random_numbers  # Add the random numbers to the DataFrame


In [3]:
holding_df = pd.read_csv("data/LargeHoldingCompanies.csv")

# Get Bank Names
df = pd.merge(df,
         holding_df,
         left_on='Random_Number', right_on='Rank', how='left')

In [4]:
df

,Student,GTID,Random_Number,Rank,Name,RSSDID,Location,TotalAsset
0,Glenn Matlin,903738044,9,9,TRUIST FINANCIAL CORPORATION ...,1074156,"CHARLOTTE, NC",5.545490e+08
1,Nikita Tatarinov,903914813,9,9,TRUIST FINANCIAL CORPORATION ...,1074156,"CHARLOTTE, NC",5.545490e+08
2,Michael Galarnyk,903918895,17,17,HSBC NORTH AMERICA HOLDINGS INC. ...,3232316,"NEW YORK, NY",2.236565e+08
3,Mitchell Kramer,903176941,1,1,JPMORGAN CHASE & CO. ...,1039502,"NEW YORK, NY",3.868240e+09
4,Nicholas Meyer,903444783,16,16,AMERICAN EXPRESS COMPANY ...,1275216,"NEW YORK, NY",2.449040e+08


## Section 1: Current Information about the bank

This section should be brief. One paragraph.
1. What are the various business units of the bank (for example, is the bank a pure commercial bank or does it engage in investment banking and other financial businesses?). You can get this information from the firm’s 10-K.

## Section 2: Understand the Ownership Structure

Analyze (see yahoo finance or Bloomberg or the company’s investor website for the ownership structure). The data needs to be extracted using Python from the relevant URL. This section should be brief. One paragraph with data tables / graphs in the appendix.
1. Institutional Ownership
2. Non-Institutional Ownership
3. Insider Ownership
4. Are the insiders buying or selling the stock recently?

## Section 3: How do the markets view the bank

This section should be a short summary with the corresponding tables in the Appendix
1. Stock Return, Beta, Volatility (compute using Python, translate your previous Rcode into Python)
2. Short-interest in the bank’s stock. days it takes to cover the short-interest (from the exchange or other sources)
3. CDS prices (from bloomberg or some other source). Can show a graph using this data
4. Credit ratings
5. Analyst opinions

## Section 4: Understanding Bank Call Reports

To help with section 5-8 below, first do the following
1. Download the PDF format of the quarterly FRY9-C (as of Dec 2022) for your bank by following your bank link
* https://www.ffiec.gov/npw/Institution/TopHoldings
2. Browse through the statement (you can use control+f to find some of the items) to get the relevant item number for some of the following variables
3. In order to understand the variables, you can see FDIC performance report and description of the ratios.
* https://www.federalreserve.gov/boarddocs/supmanual/bhcpr/UsersGuide13/default.htm
4. Now download the Q4 (as of Dec 12/31) FRY9-C for for the bank for the last five years in CSV format (not manually, make a call inside your python program) by following the bank URL
5. Keep only the variables required for sections 5-8. Stack the quarterly data
6. Plot the variables for the past five years
7. Comment on the patterns that you observe in the data. 1-2 lines per variable. Discuss in the group first.

## Section 5: Performance and Financial Position of the Bank: Very Important

From the FRY9-C (10-K when needed), compute and graph these items over the past 5 years.
1. earnings
2. net interest margins
3. non-interest or fee income
4. revenues, expenses
5. loan mix
6. past due and losses
7. losses and loan reserves
8. liquidity and funding
9. capitalization ratios
10. growth rates
11. asset composition
12. liability composition
13. liquidity and funding (important)
14. maturity gap
15. interest rate sensitivity
16. what is the major source of the bank performance?
17. how did the bank perform compared to the peers? You can get this information from the BHC Performance Report

## Section 6: Risk Profile of the Bank from Management Discussion and Analysis (MD&A) section in 10-K or 10-Q

1. Download the latest 10-K for the bank. If it is not in 2023, you may need to check latest 10-Q.
2. Go to the Management Discussion of Risk Management in the Management Discussion and Analysis (MD&A) section
3. Based on MD&A discussion,
    
    1). Is there a Asset Liability Committe (ALCO)? Describe the framework that the bank
is using for Asset Liability Management (ALM) within the bank
    
    2). Interest Rate Risk
        * How is the bank measuring and managing interest rate risk for ALM?
        * Do they discuss duration and convexity in the context of ALM?
4. How does the bank discuss Liquidity Risk Management?
    * What are some tools and metrics that the bank is using for liquidity risk management?
    * Di they discuss deposit convexity after SVB crisis? (mostly from their 10-Q)
5. Does the bank discuss how Basel III regulations affect their ALM strategies?
6. Describe the banks ALM Models and Assumptions if they disclosed it? How do they validate their ALM models?
7. How is the bank modeling deposits and loan book? What factors are they considering? Is there a scenario analysis?
8. How is the bank adapting their ALM strategy to the rising interest rate environments (may need to see latest 10-Q)
9. Do they discuss any other strategic considerations for ALM strategy and how it fits in the broad risk management framework?

## Section 7: Risk Profile of the Bank from FRY9-C and 10-K / 10-Q

Analyze the risk profile of the bank (under the same headings as the bank risks lecture).
1. Pay special attention to operational risk (fraud etc,)
2. Asset Side Risks
3. Liability side risks
4. Off-balance-sheet risks
5. Regulatory Risk
6. Exposure to real estate markets

## Section 8: Off-balance sheet activities of the Bank from FRY9-C and 10-K / 10-Q

1. Off balance sheet growth over years
2. Components of off-balance sheet
3. Bank’s Derivative Usage. What are the biggest components?
4. Securitization activities
5. Level 1, Level 2 and Level 3 assets of the bank
6. Value at Risk, Expected Shortfall and other Risk Measures noted in the 10-K/10-Q

## Additional Comments

Note:
1. You are expected to learn some new material through this assignment, in addition to the material covered through lectures in the class
2. Present the information in a graphical format to the extent possible. Use Python to generate graphs
3. Part of the task is to decide which financial information is relevant and which is not.
4. Focus on both income statement and balance sheet

## Resources

1. FDIC, NIC and FFIEC websites
2. <b>important </b>https://www.federalreserve.gov/boarddocs/supmanual/bhcpr/UsersGuide13/0313.pdf
3. http://www.federalreserve.gov/boarddocs/supmanual/default.htm#bhcuser
4. https://www.fdic.gov/search/help.html
5. http://www.ffiec.gov/nicpubweb/nicweb/nichome.aspx
6. See the Bank Holding Company Performance Report (BHCPR) at
    * https://www.ffiec.gov/npw/Institution/TopHoldings.
    * Click on your respective bank and then select BHCPR